# Import

In [44]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score, classification_report, make_scorer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder, Normalizer, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split, cross_validate, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier
import seaborn as sns
import datetime
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from tqdm import tqdm
import os, gc

pd.set_option('display.max_rows', 500)
random_state = 42
np.random.seed(random_state)
# data_filepath = Path('./data')
data_filepath = Path('/kaggle/input/child-mind-institute-problematic-internet-use')
KAPPA_SCORER = make_scorer(
    cohen_kappa_score,
    greater_is_better=True,
    weights='quadratic',
)

# Cudf parse time series

In [45]:
import cudf
parquet_partitions = list((data_filepath / 'series_train.parquet').glob('id=*'))
len(parquet_partitions)

996

In [46]:
def convert_ts_to_row(ts):
    agg_cols = ['X', 'Y', 'Z', 'enmo', 'anglez', 'light', 'battery_voltage']
    onerow = ts[agg_cols].astype(float).agg(['min', 'max', 'std', 'mean', 'median']).unstack().to_frame().T
    onerow.columns = ['_'.join(c) for c in onerow.columns]
    return onerow

def handle_parquet_partition(partition_filepath): 
    user_id = str(partition_filepath).split('=')[-1]
#     ts_df = pd.read_parquet(partition_filepath)
    ts_df = cudf.read_parquet(partition_filepath)
    ts_onerow = convert_ts_to_row(ts_df)
    ts_onerow['id'] = user_id
    
    ts_df['day_hour'] = ts_df.time_of_day.apply(lambda x: (x // 10**9) // 3600)
    ts_df['week'] = ts_df['relative_date_PCIAT'] // 7 + 1
    ts_df['month'] = ts_df['relative_date_PCIAT'] // 30 + 1
    
    # Количество периодов без движения
    # Count of periods without activity
    ts_onerow['total_inactivity_periods'] = ts_df[ts_df.enmo == 0].enmo.count()
    
    # Среднее количество часов активности в день
    # Avg hours of activity per day
    cnt_of_active_hours = ts_df[(ts_df['non-wear_flag'] == 0) & (ts_df.enmo > 0)].drop_duplicates(['relative_date_PCIAT', 'day_hour']).day_hour.count()
    cnt_of_days = len(ts_df['relative_date_PCIAT'].unique())
    ts_onerow['avg_active_hours_per_day'] = (cnt_of_active_hours or 1) / (cnt_of_days or 1)
    
    # weekend flag
    ts_onerow['weekend_flag'] = ts_df.weekday.isin([6,7]).astype(int)
    
    # Hour of pick activity
    # Пиковое время активности для клиента
    peak_hour_of_activity = ts_df[(ts_df['non-wear_flag'] == 0) & (ts_df.enmo > 0)].groupby([ts_df['day_hour']]).enmo.count()
    ts_onerow['peak_hour_of_activity'] = peak_hour_of_activity.sort_values(ascending=False).index[0]
    
    activity_mask = (ts_df['non-wear_flag'] == 0) & (ts_df.enmo > 0)

    # Relation of activity in weekday and weekend
    # Соотношение активности будни/выходные
    weekday_activity = ts_df[activity_mask & (~ts_df.weekday.isin([6,7]))].drop_duplicates().enmo.count()
    weekend_activity = ts_df[activity_mask & (ts_df.weekday.isin([6,7]))].drop_duplicates().enmo.count()
    ts_onerow['weekday_weekend_activity_ratio'] = (weekday_activity or 1) / (weekend_activity or 1)
    
    # Activity in differentn parts of day
    # Активность в разные части дня
    night_mask = (ts_df.day_hour > 0) & (ts_df.day_hour <= 6)
    morning_mask = (ts_df.day_hour > 6) & (ts_df.day_hour <= 12)
    day_mask = (ts_df.day_hour > 12) & (ts_df.day_hour <= 18)
    evening_mask = (ts_df.day_hour > 18) & ((ts_df.day_hour <= 23) | (ts_df.day_hour == 0))
    
    ts_onerow['night_activity'] = ts_df[activity_mask & night_mask].drop_duplicates().enmo.count()
    ts_onerow['morning_activity'] = ts_df[activity_mask & morning_mask].drop_duplicates().enmo.count()
    ts_onerow['day_activity'] = ts_df[activity_mask & day_mask].drop_duplicates().enmo.count()
    ts_onerow['evening_activity'] = ts_df[activity_mask & evening_mask].drop_duplicates().enmo.count()
    
    # Total time of device wear
    # Общее время ношения устройства
    ts_onerow['cnt_of_device_wearing_hours'] = ts_df[(ts_df['non-wear_flag'] == 0)].drop_duplicates(['relative_date_PCIAT', 'day_hour']).day_hour.count()

    # Relation of wearing / no wearing device
    # Соотношение ношения/не ношения устройства
    cnt_of_device_wearing_hours = ts_df[(ts_df['non-wear_flag'] == 0)].drop_duplicates(['relative_date_PCIAT', 'day_hour']).day_hour.count()
    cnt_of_device_no_wearing_hours = ts_df[(ts_df['non-wear_flag'] == 1)].drop_duplicates(['relative_date_PCIAT', 'day_hour']).day_hour.count()
    ts_onerow['relation_of_wearnig_no_wearing'] = (cnt_of_device_wearing_hours or 1) / (cnt_of_device_no_wearing_hours or 1)
    
    # Before / after PCIAT test activitys
    # Активность до и после теста PCIAT
    activity_mask = (ts_df['non-wear_flag'] == 0) & (ts_df.enmo > 0)
    pciat_period_mask = ts_df['relative_date_PCIAT'] >= 0
    pciat_period_activity = ts_df[activity_mask & pciat_period_mask].drop_duplicates().enmo.count()
    pre_pciat_period_activity = ts_df[activity_mask & (~pciat_period_mask)].drop_duplicates().enmo.count()
    ts_onerow['pciat_period_activity'] = pciat_period_activity
    ts_onerow['pre_pciat_period_activity'] = pre_pciat_period_activity
    
    # Relation before / after PCIAT test activitys
    # Соотношение активность до и после теста PCIAT
    ts_onerow['relation_pre_pciat_period_activity'] = (pciat_period_activity or 1) / (pre_pciat_period_activity or 1)
    return ts_onerow

with ThreadPoolExecutor(max_workers=os.cpu_count() * 2) as executor:
    results = list(tqdm(executor.map(lambda fpath: handle_parquet_partition(fpath), parquet_partitions), total=len(parquet_partitions)))
ts_input = cudf.concat(results).to_pandas(); del results; gc.collect()

# print(ts_input.memory_usage(deep=True).sum().round() / 1024 / 1024)
ts_input

100%|██████████| 996/996 [05:03<00:00,  3.28it/s]


,X_max,X_mean,X_median,X_min,X_std,Y_max,Y_mean,Y_median,Y_min,Y_std,...,weekday_weekend_activity_ratio,night_activity,morning_activity,day_activity,evening_activity,cnt_of_device_wearing_hours,relation_of_wearnig_no_wearing,pciat_period_activity,pre_pciat_period_activity,relation_pre_pciat_period_activity
0,1.850391,-0.054638,0.015846,-1.812031,0.633126,3.580182,-0.163923,0.015846,-2.631380,0.513286,...,2.211408,2774,12428,19762,13853,230,230.000000,49263,0,49263.0
0,1.928769,0.113277,0.068143,-1.807955,0.507897,3.234613,0.093139,0.068143,-2.887664,0.541129,...,2.998863,63927,73024,79628,60441,457,17.576923,288450,0,288450.0
0,1.021510,-0.499738,0.018467,-1.903281,0.454021,1.016589,0.046381,0.018467,-3.150104,0.510668,...,1918.550000,7653,10960,14441,5231,112,112.000000,38391,0,38391.0
0,5.908000,0.007430,0.022344,-1.684624,0.586100,2.083693,0.007583,0.022344,-2.405738,0.542189,...,1.717158,19105,80538,83177,35178,527,527.000000,220424,0,220424.0
0,3.231563,0.086653,0.024135,-1.675859,0.509845,1.033620,-0.115162,0.024135,-1.071042,0.494897,...,1.382613,1442,1704,5582,5743,122,122.000000,14553,0,14553.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1.491908,-0.067798,0.345450,-1.407426,0.591072,3.059684,0.006292,0.345450,-1.014350,0.481551,...,2.825436,85054,94607,97704,75336,548,109.600000,365727,0,365727.0
0,1.353594,0.097154,0.120130,-1.064844,0.269882,0.994583,-0.356072,0.120130,-1.012995,0.530971,...,57.000000,10,795,340,15,42,42.000000,1160,0,1160.0
0,0.999923,-0.147508,0.002575,-1.508058,0.478085,1.043029,-0.047232,0.002575,-2.958281,0.499995,...,2.985063,48898,64654,65096,46201,445,3.156028,232381,0,232381.0
0,1.004674,-0.441574,-0.068216,-1.073320,0.502446,0.981576,-0.080691,-0.068216,-1.455156,0.457471,...,4.476457,8400,8624,12199,8665,57,57.000000,39195,0,39195.0


# Data

In [2]:
!du -hs $data_filepath/*
train_df = pd.read_csv(data_filepath / 'train.csv')
test_df = pd.read_csv(data_filepath / 'test.csv')
train_df.shape, test_df.shape

12K	/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv
4.0K	/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv
8.0M	/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet
6.3G	/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet
8.0K	/kaggle/input/child-mind-institute-problematic-internet-use/test.csv
924K	/kaggle/input/child-mind-institute-problematic-internet-use/train.csv


((3960, 82), (20, 59))

In [3]:
parquet_partitions = list((data_filepath / 'series_train.parquet').glob('id=*'))
len(parquet_partitions), len(set(str(c).split('=')[1] for c in parquet_partitions) & set(train_df.id))

(996, 996)

# Cpu parse time series

In [8]:
%%time
def parse_time_of_day(nanoseconds):
    seconds = nanoseconds // 10**9
    nanoseconds_remainder = nanoseconds % 10**9
    time_of_day = datetime.timedelta(seconds=seconds)
    hours, remainder = divmod(time_of_day.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
#     formatted_time = f"{hours:02}:{minutes:02}:{seconds:02}.{nanoseconds_remainder:09}"
    return hours, minutes, seconds, nanoseconds_remainder

def convert_ts_to_row(ts):
    agg_cols = ['X', 'Y', 'Z', 'enmo', 'anglez', 'light', 'battery_voltage']
    onerow = ts[agg_cols].agg(['min', 'max', 'std', 'mean', 'median']).unstack().to_frame().T
    onerow.columns = ['_'.join(c) for c in onerow.columns]
    return onerow

def handle_parquet_partition(partition_filepath):
    user_id = str(partition_filepath).split('=')[-1]
    ts_df = pd.read_parquet(partition_filepath)
    ts_onerow = convert_ts_to_row(ts_df)
    ts_onerow['id'] = user_id
    
    ts_df['day_hour'] = ts_df.time_of_day.apply(lambda x: parse_time_of_day(x)[0])
    ts_df['week'] = ts_df['relative_date_PCIAT'] // 7 + 1
    ts_df['month'] = ts_df['relative_date_PCIAT'] // 30 + 1
    
    # Количество периодов без движения
    # Count of periods without activity
    ts_onerow['total_inactivity_periods'] = ts_df[ts_df.enmo == 0].enmo.count()
    
    # Среднее количество часов активности в день
    # Avg hours of activity per day
    cnt_of_active_hours = ts_df[(ts_df['non-wear_flag'] == 0) & (ts_df.enmo > 0)].drop_duplicates(['relative_date_PCIAT', 'day_hour']).day_hour.count()
    cnt_of_days = len(ts_df['relative_date_PCIAT'].unique())
    ts_onerow['avg_active_hours_per_day'] = (cnt_of_active_hours or 1) / (cnt_of_days or 1)
    
    # weekend flag
    ts_onerow['weekend_flag'] = ts_df.weekday.isin([6,7]).astype(int)
    
    # Hour of pick activity
    # Пиковое время активности для клиента
    peak_hour_of_activity = ts_df[(ts_df['non-wear_flag'] == 0) & (ts_df.enmo > 0)].groupby([ts_df['day_hour']]).enmo.count()
    ts_onerow['peak_hour_of_activity'] = peak_hour_of_activity.sort_values(ascending=False).index[0]
    
    activity_mask = (ts_df['non-wear_flag'] == 0) & (ts_df.enmo > 0)

    # Relation of activity in weekday and weekend
    # Соотношение активности будни/выходные
    weekday_activity = ts_df[activity_mask & (~ts_df.weekday.isin([6,7]))].drop_duplicates().enmo.count()
    weekend_activity = ts_df[activity_mask & (ts_df.weekday.isin([6,7]))].drop_duplicates().enmo.count()
    ts_onerow['weekday_weekend_activity_ratio'] = (weekday_activity or 1) / (weekend_activity or 1)
    
    # Activity in differentn parts of day
    # Активность в разные части дня
    night_mask = (ts_df.day_hour > 0) & (ts_df.day_hour <= 6)
    morning_mask = (ts_df.day_hour > 6) & (ts_df.day_hour <= 12)
    day_mask = (ts_df.day_hour > 12) & (ts_df.day_hour <= 18)
    evening_mask = (ts_df.day_hour > 18) & ((ts_df.day_hour <= 23) | (ts_df.day_hour == 0))
    
    ts_onerow['night_activity'] = ts_df[activity_mask & night_mask].drop_duplicates().enmo.count()
    ts_onerow['morning_activity'] = ts_df[activity_mask & morning_mask].drop_duplicates().enmo.count()
    ts_onerow['day_activity'] = ts_df[activity_mask & day_mask].drop_duplicates().enmo.count()
    ts_onerow['evening_activity'] = ts_df[activity_mask & evening_mask].drop_duplicates().enmo.count()
    
    # Total time of device wear
    # Общее время ношения устройства
    ts_onerow['cnt_of_device_wearing_hours'] = ts_df[(ts_df['non-wear_flag'] == 0)].drop_duplicates(['relative_date_PCIAT', 'day_hour']).day_hour.count()

    # Relation of wearing / no wearing device
    # Соотношение ношения/не ношения устройства
    cnt_of_device_wearing_hours = ts_df[(ts_df['non-wear_flag'] == 0)].drop_duplicates(['relative_date_PCIAT', 'day_hour']).day_hour.count()
    cnt_of_device_no_wearing_hours = ts_df[(ts_df['non-wear_flag'] == 1)].drop_duplicates(['relative_date_PCIAT', 'day_hour']).day_hour.count()
    ts_onerow['relation_of_wearnig_no_wearing'] = (cnt_of_device_wearing_hours or 1) / (cnt_of_device_no_wearing_hours or 1)
    
    # Before / after PCIAT test activitys
    # Активность до и после теста PCIAT
    activity_mask = (ts_df['non-wear_flag'] == 0) & (ts_df.enmo > 0)
    pciat_period_mask = ts_df['relative_date_PCIAT'] >= 0
    pciat_period_activity = ts_df[activity_mask & pciat_period_mask].drop_duplicates().enmo.count()
    pre_pciat_period_activity = ts_df[activity_mask & (~pciat_period_mask)].drop_duplicates().enmo.count()
    ts_onerow['pciat_period_activity'] = pciat_period_activity
    ts_onerow['pre_pciat_period_activity'] = pre_pciat_period_activity
    
    # Relation before / after PCIAT test activitys
    # Соотношение активность до и после теста PCIAT
    ts_onerow['relation_pre_pciat_period_activity'] = (pciat_period_activity or 1) / (pre_pciat_period_activity or 1)
    return ts_onerow

parquet_partitions = parquet_partitions[:50]
with ThreadPoolExecutor(max_workers=os.cpu_count() * 2) as executor:
    results = list(tqdm(executor.map(lambda fpath: handle_parquet_partition(fpath), parquet_partitions), total=len(parquet_partitions)))
ts_input = pd.concat(results); del results; gc.collect()

print(ts_input.memory_usage(deep=True).sum().round() / 1024 / 1024)
ts_input

100%|██████████| 50/50 [01:06<00:00,  1.34s/it]

0.015497207641601562
CPU times: user 1min 41s, sys: 9.68 s, total: 1min 50s
Wall time: 1min 7s


,X_min,X_max,X_std,X_mean,X_median,Y_min,Y_max,Y_std,Y_mean,Y_median,...,weekday_weekend_activity_ratio,night_activity,morning_activity,day_activity,evening_activity,cnt_of_device_wearing_hours,relation_of_wearnig_no_wearing,pciat_period_activity,pre_pciat_period_activity,relation_pre_pciat_period_activity
0,-1.812031,1.850391,0.633126,-0.054638,0.015846,-2.631380,3.580182,0.513286,-0.163923,-0.141810,...,2.211408,2774,12428,19762,13853,230,230.000000,49263,0,49263.000000
0,-1.807955,1.928769,0.507897,0.113277,0.094074,-2.887664,3.234613,0.541129,0.093139,0.068143,...,2.998863,63927,73024,79628,60441,457,17.576923,288450,0,288450.000000
0,-1.903281,1.021510,0.454021,-0.499738,-0.644505,-3.150104,1.016589,0.510668,0.046381,0.088542,...,1918.550000,7653,10960,14441,5231,112,112.000000,38391,0,38391.000000
0,-1.684624,5.908000,0.586100,0.007430,0.022344,-2.405738,2.083693,0.542189,0.007583,0.009674,...,1.717158,19105,80538,83177,35178,527,527.000000,220424,0,220424.000000
0,-1.675859,3.231563,0.509845,0.086653,0.053034,-1.071042,1.033620,0.494897,-0.115162,-0.087422,...,1.382613,1442,1704,5582,5743,122,122.000000,14553,0,14553.000000
0,-2.072290,1.960013,0.575908,0.031407,0.007387,-2.814025,2.246533,0.496323,0.066393,0.037431,...,2.481995,59074,73963,80220,60297,456,4.470588,284862,0,284862.000000
0,-1.541927,0.981914,0.513974,-0.344942,-0.479531,-2.442031,0.997865,0.406852,0.017493,0.065996,...,13699.000000,2446,4253,4288,2278,25,25.000000,13699,0,13699.000000
0,-2.284167,2.065781,0.618117,0.135211,0.260742,-3.849088,3.578021,0.519876,0.013214,0.007083,...,3.251249,4214,26075,35994,26419,249,249.000000,95279,0,95279.000000
0,-2.004243,1.113939,0.468443,-0.489235,-0.640694,-1.691234,1.546270,0.451332,-0.018764,-0.018557,...,4.211699,11839,101812,99023,38811,630,630.000000,252241,0,252241.000000
0,-2.195360,1.565700,0.500070,-0.307811,-0.417052,-3.611804,1.610933,0.429974,0.023515,-0.011621,...,2.896488,79188,90616,90143,70434,517,12.309524,343756,0,343756.000000


In [4]:
%%time
def parse_time_of_day(nanoseconds):
    seconds = nanoseconds // 10**9
    nanoseconds_remainder = nanoseconds % 10**9
    time_of_day = datetime.timedelta(seconds=seconds)
    hours, remainder = divmod(time_of_day.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
#     formatted_time = f"{hours:02}:{minutes:02}:{seconds:02}.{nanoseconds_remainder:09}"
    return hours, minutes, seconds, nanoseconds_remainder

def convert_ts_to_row(ts):
    agg_cols = ['X', 'Y', 'Z', 'enmo', 'anglez', 'light', 'battery_voltage']
    onerow = ts.agg(['min', 'max', 'std', 'mean', 'median']).unstack().to_frame().T
    onerow.columns = ['_'.join(c) for c in onerow.columns]
    return onerow

def handle_parquet_partition(partition_filepath):
    user_id = str(partition_filepath).split('=')[-1]
    ts_df = pd.read_parquet(partition_filepath)
    ts_onerow = convert_ts_to_row(ts_df.copy())
    ts_onerow['id'] = user_id
    
    ts_df['day_hour'] = ts_df.time_of_day.apply(lambda x: parse_time_of_day(x)[0])
    ts_df['week'] = ts_df['relative_date_PCIAT'] // 7 + 1
    ts_df['month'] = ts_df['relative_date_PCIAT'] // 30 + 1
    
    # Количество периодов без движения
    # Count of periods without activity
    ts_onerow['total_inactivity_periods'] = ts_df[ts_df.enmo == 0].enmo.count()
    
    # Среднее количество часов активности в день
    # Avg hours of activity per day
    cnt_of_active_hours = ts_df[(ts_df['non-wear_flag'] == 0) & (ts_df.enmo > 0)].drop_duplicates(['relative_date_PCIAT', 'day_hour']).day_hour.count()
    cnt_of_days = len(ts_df['relative_date_PCIAT'].unique())
    ts_onerow['avg_active_hours_per_day'] = (cnt_of_active_hours or 1) / (cnt_of_days or 1)
    
    # weekend flag
    ts_onerow['weekend_flag'] = ts_df.weekday.isin([6,7]).astype(int)
    
    # Hour of pick activity
    # Пиковое время активности для клиента
    peak_hour_of_activity = ts_df[(ts_df['non-wear_flag'] == 0) & (ts_df.enmo > 0)].groupby([ts_df['day_hour']]).enmo.count()
    ts_onerow['peak_hour_of_activity'] = peak_hour_of_activity.sort_values(ascending=False).index[0]
    
    activity_mask = (ts_df['non-wear_flag'] == 0) & (ts_df.enmo > 0)

    # Relation of activity in weekday and weekend
    # Соотношение активности будни/выходные
    weekday_activity = ts_df[activity_mask & (~ts_df.weekday.isin([6,7]))].drop_duplicates().enmo.count()
    weekend_activity = ts_df[activity_mask & (ts_df.weekday.isin([6,7]))].drop_duplicates().enmo.count()
    ts_onerow['weekday_weekend_activity_ratio'] = (weekday_activity or 1) / (weekend_activity or 1)
    
    # Activity in differentn parts of day
    # Активность в разные части дня
    night_mask = (ts_df.day_hour > 0) & (ts_df.day_hour <= 6)
    morning_mask = (ts_df.day_hour > 6) & (ts_df.day_hour <= 12)
    day_mask = (ts_df.day_hour > 12) & (ts_df.day_hour <= 18)
    evening_mask = (ts_df.day_hour > 18) & ((ts_df.day_hour <= 23) | (ts_df.day_hour == 0))
    
    ts_onerow['night_activity'] = ts_df[activity_mask & night_mask].drop_duplicates().enmo.count()
    ts_onerow['morning_activity'] = ts_df[activity_mask & morning_mask].drop_duplicates().enmo.count()
    ts_onerow['day_activity'] = ts_df[activity_mask & day_mask].drop_duplicates().enmo.count()
    ts_onerow['evening_activity'] = ts_df[activity_mask & evening_mask].drop_duplicates().enmo.count()
    
    # Total time of device wear
    # Общее время ношения устройства
    ts_onerow['cnt_of_device_wearing_hours'] = ts_df[(ts_df['non-wear_flag'] == 0)].drop_duplicates(['relative_date_PCIAT', 'day_hour']).day_hour.count()

    # Relation of wearing / no wearing device
    # Соотношение ношения/не ношения устройства
    cnt_of_device_wearing_hours = ts_df[(ts_df['non-wear_flag'] == 0)].drop_duplicates(['relative_date_PCIAT', 'day_hour']).day_hour.count()
    cnt_of_device_no_wearing_hours = ts_df[(ts_df['non-wear_flag'] == 1)].drop_duplicates(['relative_date_PCIAT', 'day_hour']).day_hour.count()
    ts_onerow['relation_of_wearnig_no_wearing'] = (cnt_of_device_wearing_hours or 1) / (cnt_of_device_no_wearing_hours or 1)
    
    # Before / after PCIAT test activitys
    # Активность до и после теста PCIAT
    activity_mask = (ts_df['non-wear_flag'] == 0) & (ts_df.enmo > 0)
    pciat_period_mask = ts_df['relative_date_PCIAT'] >= 0
    pciat_period_activity = ts_df[activity_mask & pciat_period_mask].drop_duplicates().enmo.count()
    pre_pciat_period_activity = ts_df[activity_mask & (~pciat_period_mask)].drop_duplicates().enmo.count()
    ts_onerow['pciat_period_activity'] = pciat_period_activity
    ts_onerow['pre_pciat_period_activity'] = pre_pciat_period_activity
    
    # Relation before / after PCIAT test activitys
    # Соотношение активность до и после теста PCIAT
    ts_onerow['relation_pre_pciat_period_activity'] = (pciat_period_activity or 1) / (pre_pciat_period_activity or 1)
    return ts_onerow

with ThreadPoolExecutor(max_workers=os.cpu_count() * 2) as executor:
    results = list(tqdm(executor.map(lambda fpath: handle_parquet_partition(fpath), parquet_partitions), total=len(parquet_partitions)))
ts_train = pd.concat(results); del results
ts_train

 26%|██▌       | 254/996 [04:34<13:22,  1.08s/it]


KeyboardInterrupt: 

In [5]:
%%time
ts_train.to_parquet('ts_train.parquet')
!du -hs 'ts_train.parquet'

NameError: name 'ts_train' is not defined

In [2]:
%%time
ts_train = pd.read_parquet(data_filepath.parent / 'ts-train-parquet')
train_df = pd.read_csv(data_filepath / 'train.csv')
print(train_df.shape, ts_train.shape)
train_df = pd.merge(train_df, ts_train, on='id', how='left')
train_df.shape

(3960, 82) (996, 80)
CPU times: user 96.4 ms, sys: 49.3 ms, total: 146 ms
Wall time: 310 ms


(3960, 161)

In [3]:
def feature_engineering(df):
    # season_cols = [col for col in df.columns if 'Season' in col]
    # df = df.drop(season_cols, axis=1)
    pciat_cols = [c for c in df.columns if c.startswith('PCIAT')]
    df = df.drop(columns=pciat_cols, errors='ignore')
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']

    df['Age_Weight'] = df['Basic_Demos-Age'] * df['Physical-Weight']
    df['Sex_BMI'] = df['Basic_Demos-Sex'] * df['Physical-BMI']
    df['Sex_HeartRate'] = df['Basic_Demos-Sex'] * df['Physical-HeartRate']
    df['Age_WaistCirc'] = df['Basic_Demos-Age'] * df['Physical-Waist_Circumference']
    df['BMI_FitnessMaxStage'] = df['Physical-BMI'] * df['Fitness_Endurance-Max_Stage']
    df['Weight_GripStrengthDominant'] = df['Physical-Weight'] * df['FGC-FGC_GSD']
    df['Weight_GripStrengthNonDominant'] = df['Physical-Weight'] * df['FGC-FGC_GSND']
    df['HeartRate_FitnessTime'] = df['Physical-HeartRate'] * (df['Fitness_Endurance-Time_Mins'] + df['Fitness_Endurance-Time_Sec'])
    df['Age_PushUp'] = df['Basic_Demos-Age'] * df['FGC-FGC_PU']
    df['FFMI_Age'] = df['BIA-BIA_FFMI'] * df['Basic_Demos-Age']
    df['InternetUse_SleepDisturbance'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['SDS-SDS_Total_Raw']
    df['CGAS_BMI'] = df['CGAS-CGAS_Score'] * df['Physical-BMI']
    df['CGAS_FitnessMaxStage'] = df['CGAS-CGAS_Score'] * df['Fitness_Endurance-Max_Stage']
    return df

train_df = feature_engineering(train_df)
train_df = train_df.replace([np.inf, -np.inf], np.nan)

In [4]:
%%time
train_df_imputed = train_df.drop(columns='id').copy()
cat_cols = train_df_imputed.select_dtypes(include='object').columns.tolist()
num_cols = train_df_imputed.drop(columns='sii').select_dtypes(include='number').columns.tolist()
print(f'{len(cat_cols)=} {len(num_cols)=}')
impute_cols = list(set(num_cols + ['sii']))
knn_imputer = KNNImputer(n_neighbors=5)
train_df_imputed.loc[:,num_cols] = knn_imputer.fit_transform(train_df_imputed[num_cols])
train_df_imputed.loc[:,cat_cols] = train_df_imputed[cat_cols].fillna('missing')
train_df_imputed['sii'] = pd.DataFrame(KNNImputer(n_neighbors=5).fit_transform(train_df_imputed[impute_cols]), columns=impute_cols).sii
train_df_imputed['sii'] = train_df_imputed.sii.round().astype(int)
train_df_imputed.isna().sum().sum()

len(cat_cols)=10 len(num_cols)=155
CPU times: user 15.7 s, sys: 3.62 s, total: 19.4 s
Wall time: 16.9 s


0

In [5]:
train_df.sii.value_counts(dropna=False).sort_index()

sii
0.0    1594
1.0     730
2.0     378
3.0      34
NaN    1224
Name: count, dtype: int64

In [6]:
train_df_imputed.sii.value_counts(dropna=False).sort_index()

sii
0    1596
1    1777
2     378
3     209
Name: count, dtype: int64

In [7]:
cols_to_drop = ['sii', 'id']
X, y = train_df_imputed.drop(columns=cols_to_drop, errors='ignore'), train_df_imputed.sii
cat_cols = X.select_dtypes(include='object').columns.tolist()
num_cols = X.select_dtypes(include='number').columns.tolist()
X.shape, y.shape

((3960, 165), (3960,))

In [8]:
X.isna().sum().sum()

0

In [9]:
numeric_transormer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    # ('imputer', SimpleImputer(strategy='constant', fill_value=-999)),
])
category_transormer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OrdinalEncoder()),
])
cols_transformer = ColumnTransformer(transformers=[
    ('numeric', numeric_transormer, num_cols),
    ('category', category_transormer, cat_cols),
])
preproc_df_pipe = Pipeline(steps=[
    ('cols_transformer', cols_transformer),
    ('scaler', StandardScaler()), # MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler
#     ('scaler', MinMaxScaler()),
    # ('scaler', RobustScaler()),
    # ('scaler', MaxAbsScaler()),
])
preproc_df_pipe

Pipeline(steps=[('cols_transformer',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Basic_Demos-Age',
                                                   'Basic_Demos-Sex',
                                                   'CGAS-CGAS_Score',
                                                   'Physical-BMI',
                                                   'Physical-Height',
                                                   'Physical-Weight',
                                                   'Physical-Waist_Circumference',
                                                   'Physical-Diastolic_BP',
                                                   'Physical-HeartRate',
                                                   'Physical-Systolic_BP',
                                                   'Fitness_Endurance-M...
                                                   'BIA-BIA_BMC', 'BIA-BIA_BMI', ...]),
                                                 ('category',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OrdinalEncoder())]),
                                                  ['Basic_Demos-Enroll_Season',
                                                   'CGAS-Season',
                                                   'Physical-Season',
                                                   'Fitness_Endurance-Season',
                                                   'FGC-Season', 'BIA-Season',
                                                   'PAQ_A-Season',
                                                   'PAQ_C-Season', 'SDS-Season',
                                                   'PreInt_EduHx-Season'])])),
                ('scaler', StandardScaler())])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=random_state)
X_train = preproc_df_pipe.fit_transform(X_train)
X_test = preproc_df_pipe.transform(X_test)

In [11]:
%%time
clf = CatBoostClassifier()
clf.fit(
    X_train, y_train,
    logging_level='Silent', 
)
preds = clf.predict(X_test)
print(cohen_kappa_score(y_test, preds, weights='quadratic'))
print(classification_report(y_test, preds))

0.5700542049401149
              precision    recall  f1-score   support

           0       0.65      0.83      0.73       479
           1       0.75      0.66      0.70       533
           2       0.36      0.15      0.21       113
           3       0.80      0.75      0.77        63

    accuracy                           0.69      1188
   macro avg       0.64      0.60      0.60      1188
weighted avg       0.67      0.69      0.67      1188

CPU times: user 3min 12s, sys: 3.09 s, total: 3min 16s
Wall time: 51.5 s


In [12]:
%%time
_clf = CatBoostClassifier(verbose=0)
cv = StratifiedShuffleSplit(n_splits=5, test_size=.3, random_state=random_state)
_pipeline = Pipeline([('transformer', preproc_df_pipe), ('estimator', _clf)])
scoring = {
    'f1': 'f1_macro',f
    'cohen_kappa_score': KAPPA_SCORER, 
}
scores = cross_validate(_pipeline, X, y, cv=cv, scoring=scoring)
for metric, folds_score in scores.items():
    if not metric.startswith('test_'): continue
    print(metric, folds_score.mean().round(4), folds_score.std().round(4))

test_f1 0.6012 0.0135
test_cohen_kappa_score 0.5671 0.0208
CPU times: user 16min 5s, sys: 15.4 s, total: 16min 20s
Wall time: 4min 17s


In [61]:
test_df = pd.read_csv(data_filepath / 'test.csv')
test_df = pd.merge(test_df, ts_test, on='id', how='inner')
test_df = feature_engineering(test_df)
test_df['sii'] = 2
test_df.loc[:,impute_cols] = knn_imputer.transform(test_df[impute_cols])
test_df.drop(columns='sii', inplace=True)
# test_df.loc[:,num_cols] = KNNImputer(n_neighbors=5).fit_transform(test_df[num_cols])
test_df.loc[:,cat_cols] = test_df[cat_cols].fillna('missing')
_id = test_df.id
test_df = pd.DataFrame(preproc_df_pipe.transform(test_df), columns=test_df[X.columns].columns)
test_df['id'] = _id; del _id
test_df['sii'] = clf.predict(test_df[X.columns])
test_df.sii.value_counts()

sii
0    1
1    1
Name: count, dtype: int64

In [62]:
test_df[['id', 'sii']].to_csv('submission.csv', index=False)
!du -hs 'submission.csv'

4.0K	submission.csv
